# League of Legends API information and implementation

## Queue

To "queue up" in a game means to join a waiting list or a matchmaking system to play a game with other players. There are different types of queues:

Queue list = [RANKED_SOLO_5x5, RANKED_FLEX_SR, RANKED_FLEX_TT]

## Tier

Ranked system in League of Legends is divided into nine different tiers:

Tier list = [IRON, BRONZE, SILVER, GOLD, PLATINUM, DIAMOND, MASTER, GRANDMASTER, CHALLENGER]

## Division

Each of the nine tiers are further subdivided into divisions. For example, there are four divisions for each tier between Iron to Diamond.

From Master rank onwards, there is no sub-division

Division list = [I, II, III, IV]

## Regions

Riot Games provides servers to host various aspects of League of Legends icon League of Legends. These include but are not limited to logins, games, chat, the RP Store, the in-game shop, etc.

Region dict = {"BR1" : "Brazil", "EUN1" : "Europe Nordic & East", "EUW1" : "Europe West", "LA1" : "Latin America North", "LA2" : "Latin America South", "NA1" : "North America", "OCE" : "Oceania", "RU" : "Russia", "TR" : "Turkey", "JP" : "Japan", "KR" : "Republic of Korea", "PH" : "The Philippines", "SG" : "Singapore, Malaysia, & Indonesia", "TW" : "Taiwan, Hong Kong, and Macao", "TH" : "Thailand", "VN" : "Vietnam"}

### LEAGUE-EXP-V4

Link: https://developer.riotgames.com/apis#league-exp-v4

/lol/league-exp/v4/entries/{queue}/{tier}/{division}

Returns information of players who have played ranked and have a tier

Request URL example: https://la2.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/IRON/IV?page=2&...

Needed info: Queue, Tier, Division, Region, Page

### Requests to LEAGUE-EXP-V4

#### Needed Imports

In [1]:
#Import needed packages
import pandas as pd

import requests

#To limit the amount of requests per minute
from ratelimit import limits, sleep_and_retry #from https://pypi.org/project/ratelimit/

#To load the key from .env and more
from dotenv import load_dotenv
import os

%load_ext autoreload
%autoreload 2

#### Needed Information

In [19]:
#Spirit of the LoL api key
# Cargar el archivo .env
load_dotenv()

# Acceder al valor de una variable
api_key = os.getenv("API_KEY")

#Code for LAS region
region = "LA2"

#Code for queue Solo
queue = "RANKED_SOLO_5x5"

queue_list = ["RANKED_SOLO_5x5", "RANKED_FLEX_SR", "RANKED_FLEX_TT"]

#Tiers
tier_list = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "DIAMOND", "MASTER", "GRANDMASTER", "CHALLENGER"]

#Remember that from Master rank and above, only division I is valid
division_list = ["IV", "III", "II", "I"]

#### Needed Functions for league_exp_v4

In [3]:
def save_to_csv_league_exp_v4(df, queue):
    """
    Saves the dataframe obtained from league_exp_v4 into a csv file
    """
    
    df.to_csv(f'Data/league_exp_v4_{region}_{queue}.csv')  
    
    return print(f'Saved to Data/league_exp_v4_{region}_{queue}.csv')

In [4]:
#100 requests every 120 sec

time_seconds = 10

max_requests = 8

@sleep_and_retry
@limits(calls= max_requests, period= time_seconds) #period is in seconds 
def fetch_league_exp_v4(queue, tier, division, region, page, api_key):
    """
    Get LeagueEntryDTO from LEAGUE-EXP-V4 API
    """
    url = f'https://{region}.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}?page={page}&api_key={api_key}'
    
    response = requests.get(url)
    
    if response.status_code != 200:
                    raise Exception('API response: {}'.format(response.status_code))
            
    return response.json()

In [5]:
def iterate_league_exp_v4():
    """
    Get all LeagueEntryDTO from LEAGUE-EXP-V4 API for a certain tier.
    """
    #generates and empy DF to store and return the data
    complete_df = pd.DataFrame()
    
    #Loops over the Tier list
    for tier in tier_list:
    
        #Verifies tier to define possible divisions
        if tier in ["MASTER", "GRANDMASTER", "CHALLENGER"]:
            division_list = ["I"]
        else:
            division_list = ["IV", "III", "II", "I"]

        #Loops over divisions
        for division in division_list:

            #Creates page counter
            page = 1 

            #Creates not empty DF
            response_df = pd.DataFrame({'not_empty': [1]})

            #Loops over pages until it finds and empty response
            while not response_df.empty:

                    response_df = pd.DataFrame(fetch_league_exp_v4(queue, tier, division, region, page, api_key))

                    complete_df = pd.concat([complete_df,response_df], ignore_index = True)

                    print(f"Tier: {tier}, Division: {division}, Page: {page}")

                    page = page + 1

                    if response_df.empty:
                        print("Empty")
                
    return complete_df

In [ ]:
%%time

league_exp_v4_df = iterate_league_exp_v4()

save_to_csv_league_exp_v4(league_exp_v4_df, queue)

##### Nota: Feature importance, decision tree

#### Needed Functions for summoner_v4

In [22]:
#1600 requests every 60 seconds
#or 100 requests every 120 sec? check

time_seconds = 10

max_requests = 8

@sleep_and_retry
@limits(calls= max_requests, period= time_seconds) #period is in seconds 
def fetch_summoner_v4(region, summoner_name, api_key):
    """
    Get SummonerDTO from SUMMONER_V4 API
    """
    url = f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={api_key}"
    
    #try:
    
    response = requests.get(url)
        
    print(f"Summoner Name: {summoner_name}, 'API response: {response.status_code}")
        
   # except:
                        
        #print(f"Summoner Name: {summoner_name}, 'API response: {resonse.status_code}")
        
        #response = pd.DataFrame()
            
       # pass
    
    #if response.status_code != 200:
    #                raise Exception('API response: {}'.format(response.status_code))
            
    return response.json()

In [84]:
def iterate_summoner_v4(league_exp_v4_df):
    """
    Get all SummonerDTO from SUMMONER-V4 API, it takes a long time because of API limit
    """
    #Takes de series I need with the summoner name and converts it into an array to loop
    summoner_name_array = league_exp_v4_df["summonerName"].to_numpy()
    
    #generates and empy DF to store and return the data
    complete_df = pd.DataFrame()
    
    #Inicio un contador en 0
    count = 0
    
    #Loops over summoner name
    for i in range(0,1000):
              
        response_df = pd.DataFrame((fetch_summoner_v4(region, summoner_name_array[i], api_key)),index=[0])

        complete_df = pd.concat([complete_df,response_df], ignore_index = True)
            
        count = count + 1
                    
        if count % 100 == 0:
            
            print(f"Count: {count}")#Prints when we send 100 requests)
                
    return complete_df

In [47]:
def iterate_summoner_v4_lite(league_exp_v4_df, tier, max_data):
    """
    Get max_data SummonerDTO from SUMMONER-V4 API
    """
    number_summoners= len(league_exp_v4_df[league_exp_v4_df["tier"] == tier])
    
    print(f"\nNumber of Summoners in tier {tier}: {number_summoners}\n")
    
    #Takes de series I need with the summoner name and converts it into an array to loop
    summoner_name_array = league_exp_v4_df[league_exp_v4_df["tier"] == tier]["summonerName"].to_numpy()
    
    #generates and empy DF to store and return the data
    complete_df = pd.DataFrame()
    
    #Inicio un contador en 0
    count = 0
    
    #Loops over summoner name
    if number_summoners < max_data:
        for i in range(0,number_summoners):
            
            print(f"Summoner {i+1} from {number_summoners}")

            response_df = pd.DataFrame((fetch_summoner_v4(region, summoner_name_array[i], api_key)),index=[0])

            complete_df = pd.concat([complete_df,response_df], ignore_index = True)

            count = count + 1

            if count % 50 == 0:

                print(f"\nCount: {count}\n")#Prints when we send 100 requests)
    
    else:
        for i in range(0,number_summoners,int(number_summoners/max_data)):
            
            print(f"Summoner {i+1} from {number_summoners}")

            response_df = pd.DataFrame((fetch_summoner_v4(region, summoner_name_array[i], api_key)),index=[0])

            complete_df = pd.concat([complete_df,response_df], ignore_index = True)

            count = count + 1

            if count % 50 == 0:

                print(f"\nCount: {count}\n")#Prints when we send 100 requests)
                
    return complete_df

In [41]:
def save_to_csv_summoner_v4_lite(df, tier):
    """
    Saves the dataframe obtained from summoner_v4_lite into a csv file
    """
    df.to_csv(f'Data/summoner_v4_lite_{tier}.csv')  
    
    return print(f'\nSaved to Data/summoner_v4_lite_{tier}.csv\n')

In [50]:
%%time

league_exp_v4_df = pd.read_csv("Data/league_exp_v4_LA2_RANKED_SOLO_5x5.csv")

upper_tier_list = ["DIAMOND", "MASTER", "GRANDMASTER", "CHALLENGER"]

middle_tier_list = ["GOLD", "PLATINUM"]

bottom_tier_list = ["IRON", "BRONZE", "SILVER"]

max_data = 1000

for i in bottom_tier_list:
    df = iterate_summoner_v4_lite(league_exp_v4_df, i, max_data)
    save_to_csv_summoner_v4_lite(df, i)



Number of Summoners in tier IRON: 40223

Summoner 1 from 40223
Summoner Name: brandonmoraa, 'API response: 200
Summoner 41 from 40223
Summoner Name: Xaspart, 'API response: 200
Summoner 81 from 40223
Summoner Name: ZetaleonpaezGame, 'API response: 200
Summoner 121 from 40223
Summoner Name: ETSecretariaS3X0, 'API response: 200
Summoner 161 from 40223
Summoner Name: Crialsoff, 'API response: 200
Summoner 201 from 40223
Summoner Name: RumblesxBoric, 'API response: 200
Summoner 241 from 40223
Summoner Name: havokdeath, 'API response: 200
Summoner 281 from 40223
Summoner Name: Aaron4565, 'API response: 200
Summoner 321 from 40223
Summoner Name: Brandr3, 'API response: 200
Summoner 361 from 40223
Summoner Name: wed024, 'API response: 200
Summoner 401 from 40223
Summoner Name: XxJhoël wasonxX, 'API response: 200
Summoner 441 from 40223
Summoner Name: Cristhianxd1456, 'API response: 200
Summoner 481 from 40223
Summoner Name: drinfa, 'API response: 200
Summoner 521 from 40223
Summoner Name: Es

Summoner Name: kirius666, 'API response: 200
Summoner 4641 from 40223
Summoner Name: kevinn420, 'API response: 200
Summoner 4681 from 40223
Summoner Name: saguil14, 'API response: 200
Summoner 4721 from 40223
Summoner Name: NoMeLaContainer, 'API response: 200
Summoner 4761 from 40223
Summoner Name: joacotv4, 'API response: 200
Summoner 4801 from 40223
Summoner Name: 1FUTURE1, 'API response: 200
Summoner 4841 from 40223
Summoner Name: MellowSwing, 'API response: 200
Summoner 4881 from 40223
Summoner Name: Ka0ss, 'API response: 200
Summoner 4921 from 40223
Summoner Name: MatuCanobb, 'API response: 200
Summoner 4961 from 40223
Summoner Name: PeriHellioN, 'API response: 200
Summoner 5001 from 40223
Summoner Name: de1untulaso, 'API response: 200
Summoner 5041 from 40223
Summoner Name: Eikadaj, 'API response: 200
Summoner 5081 from 40223
Summoner Name: MefiistO169, 'API response: 200
Summoner 5121 from 40223
Summoner Name: ElSantuso, 'API response: 200
Summoner 5161 from 40223
Summoner Name:

Summoner Name: joelsolagato1, 'API response: 200
Summoner 9281 from 40223
Summoner Name: XxVICTORxXXX, 'API response: 200
Summoner 9321 from 40223
Summoner Name: PetaK, 'API response: 200
Summoner 9361 from 40223
Summoner Name: demiansito, 'API response: 200
Summoner 9401 from 40223
Summoner Name: rocketman, 'API response: 200
Summoner 9441 from 40223
Summoner Name: yzz0f, 'API response: 200
Summoner 9481 from 40223
Summoner Name: SoldierOffHell, 'API response: 200
Summoner 9521 from 40223
Summoner Name: gerouru0923, 'API response: 200
Summoner 9561 from 40223
Summoner Name: marcelosalts, 'API response: 200
Summoner 9601 from 40223
Summoner Name: eydaankeifer, 'API response: 200
Summoner 9641 from 40223
Summoner Name: PAP Zeus, 'API response: 200
Summoner 9681 from 40223
Summoner Name: Elliwood, 'API response: 200
Summoner 9721 from 40223
Summoner Name: imBussky, 'API response: 200
Summoner 9761 from 40223
Summoner Name: Super noob, 'API response: 200
Summoner 9801 from 40223
Summoner 

Summoner Name: ZapasoEres, 'API response: 200
Summoner 13841 from 40223
Summoner Name: Cacha Canaya, 'API response: 200
Summoner 13881 from 40223
Summoner Name: BadNey, 'API response: 200
Summoner 13921 from 40223
Summoner Name: santinokz9813, 'API response: 200
Summoner 13961 from 40223
Summoner Name: TQAlejandroG647, 'API response: 200

Count: 350

Summoner 14001 from 40223
Summoner Name: xSev7n, 'API response: 200
Summoner 14041 from 40223
Summoner Name: Amasamela, 'API response: 200
Summoner 14081 from 40223
Summoner Name: AkOuTT47, 'API response: 200
Summoner 14121 from 40223
Summoner Name: ñoño666, 'API response: 200
Summoner 14161 from 40223
Summoner Name: w4rbobby, 'API response: 200
Summoner 14201 from 40223
Summoner Name: DGxZ, 'API response: 200
Summoner 14241 from 40223
Summoner Name: lautym00, 'API response: 200
Summoner 14281 from 40223
Summoner Name: thebrunio, 'API response: 200
Summoner 14321 from 40223
Summoner Name: JOSE MERCADO, 'API response: 200
Summoner 14361 fro

Summoner Name: Letalidad22, 'API response: 200
Summoner 18401 from 40223
Summoner Name: ELGORDODRILLL, 'API response: 200
Summoner 18441 from 40223
Summoner Name: xLucasxCapitax, 'API response: 200
Summoner 18481 from 40223
Summoner Name: Aysel, 'API response: 200
Summoner 18521 from 40223
Summoner Name: EN  CULO, 'API response: 200
Summoner 18561 from 40223
Summoner Name: UnPresoDeJujuy, 'API response: 200
Summoner 18601 from 40223
Summoner Name: DURIZAR, 'API response: 200
Summoner 18641 from 40223
Summoner Name: Droguita420, 'API response: 200
Summoner 18681 from 40223
Summoner Name: mcQIYANA, 'API response: 200
Summoner 18721 from 40223
Summoner Name: diegoskar, 'API response: 200
Summoner 18761 from 40223
Summoner Name: BenjaMataHackers, 'API response: 200
Summoner 18801 from 40223
Summoner Name: Rodrix2374, 'API response: 200
Summoner 18841 from 40223
Summoner Name: DarkDemon516B, 'API response: 200
Summoner 18881 from 40223
Summoner Name: bamsxone, 'API response: 200
Summoner 18

...

Summoner Name: PorSiempreHierro, 'API response: 200
Summoner 91763 from 194785
Summoner Name: miguexi, 'API response: 200
Summoner 91957 from 194785
Summoner Name: LeoUbillus, 'API response: 200
Summoner 92151 from 194785
Summoner Name: Leanly, 'API response: 200
Summoner 92345 from 194785
Summoner Name: Mastershyfu, 'API response: 200
Summoner 92539 from 194785
Summoner Name: AlbertoFernández, 'API response: 200
Summoner 92733 from 194785
Summoner Name: GengisSW, 'API response: 200
Summoner 92927 from 194785
Summoner Name: Ourienaor, 'API response: 200
Summoner 93121 from 194785
Summoner Name: Malole, 'API response: 200
Summoner 93315 from 194785
Summoner Name: StrongTotipret, 'API response: 200
Summoner 93509 from 194785
Summoner Name: Ivie, 'API response: 200
Summoner 93703 from 194785
Summoner Name: Assasinxd23, 'API response: 200
Summoner 93897 from 194785
Summoner Name: ZYKKEK ADC, 'API response: 200
Summoner 94091 from 194785
Summoner Name: ll Yuna ll, 'API response: 200
Summone

Summoner Name: Roocioo, 'API response: 200
Summoner 113491 from 194785
Summoner Name: TitoYatusae, 'API response: 200
Summoner 113685 from 194785
Summoner Name: Arcabuz, 'API response: 200
Summoner 113879 from 194785
Summoner Name: GUARDA CON TOBI, 'API response: 200
Summoner 114073 from 194785
Summoner Name: romantiic, 'API response: 200
Summoner 114267 from 194785
Summoner Name: xX Leeyends Xx, 'API response: 200
Summoner 114461 from 194785
Summoner Name: CjvsCL, 'API response: 200
Summoner 114655 from 194785
Summoner Name: AlfreMontesDeOca, 'API response: 200
Summoner 114849 from 194785
Summoner Name: clau27, 'API response: 200
Summoner 115043 from 194785
Summoner Name: voltronuwu, 'API response: 200
Summoner 115237 from 194785
Summoner Name: Z3ERØ, 'API response: 200
Summoner 115431 from 194785
Summoner Name: cualquierad, 'API response: 200
Summoner 115625 from 194785
Summoner Name: Yato Sensei, 'API response: 200
Summoner 115819 from 194785
Summoner Name: conmor, 'API response: 20

Summoner Name: ramonalamonax, 'API response: 200
Summoner 135025 from 194785
Summoner Name: Loboarhe, 'API response: 200
Summoner 135219 from 194785
Summoner Name: camkz, 'API response: 200
Summoner 135413 from 194785
Summoner Name: Y4METE KUD4SS4I, 'API response: 200
Summoner 135607 from 194785
Summoner Name: Yoyito0, 'API response: 200

Count: 700

Summoner 135801 from 194785
Summoner Name: XeVeX, 'API response: 200
Summoner 135995 from 194785
Summoner Name: TP J o b s, 'API response: 200
Summoner 136189 from 194785
Summoner Name: Cry Me A Rıven, 'API response: 200
Summoner 136383 from 194785
Summoner Name: FurryKrissMain, 'API response: 200
Summoner 136577 from 194785
Summoner Name: Â R W E N, 'API response: 200
Summoner 136771 from 194785
Summoner Name: Nachinh0, 'API response: 200
Summoner 136965 from 194785
Summoner Name: 1Manu1, 'API response: 200
Summoner 137159 from 194785
Summoner Name: nojom, 'API response: 200
Summoner 137353 from 194785
Summoner Name: KashmirInnuendo, 'API

Summoner Name: Desotto, 'API response: 200
Summoner 156559 from 194785
Summoner Name: Leeme, 'API response: 200
Summoner 156753 from 194785
Summoner Name: Elesh Norn, 'API response: 200
Summoner 156947 from 194785
Summoner Name: El Messiratatata, 'API response: 404
Summoner 157141 from 194785
Summoner Name: SG Mado, 'API response: 200
Summoner 157335 from 194785
Summoner Name: bbynose, 'API response: 200
Summoner 157529 from 194785
Summoner Name: Gvtøxic, 'API response: 200
Summoner 157723 from 194785
Summoner Name: El GuRu, 'API response: 200
Summoner 157917 from 194785
Summoner Name: Sµndrømę, 'API response: 200
Summoner 158111 from 194785
Summoner Name: GinhowXDe, 'API response: 200
Summoner 158305 from 194785
Summoner Name: THE NEME, 'API response: 200
Summoner 158499 from 194785
Summoner Name: PumaRodriguez44, 'API response: 200
Summoner 158693 from 194785
Summoner Name: gordasusana, 'API response: 200
Summoner 158887 from 194785
Summoner Name: sSavage, 'API response: 200
Summoner

Summoner Name: rojasus, 'API response: 200
Summoner 178093 from 194785
Summoner Name: R7 KĄ ZHÎX, 'API response: 200
Summoner 178287 from 194785
Summoner Name: yukitoki, 'API response: 200
Summoner 178481 from 194785
Summoner Name: Cngointhu, 'API response: 200
Summoner 178675 from 194785
Summoner Name: Listahda, 'API response: 200
Summoner 178869 from 194785
Summoner Name: Yo soy EL, 'API response: 200
Summoner 179063 from 194785
Summoner Name: Crive55, 'API response: 200
Summoner 179257 from 194785
Summoner Name: StepFears1, 'API response: 200
Summoner 179451 from 194785
Summoner Name: Goku Fernandez, 'API response: 200
Summoner 179645 from 194785
Summoner Name: Zorrito126A, 'API response: 200
Summoner 179839 from 194785
Summoner Name: Aleph Zer0, 'API response: 200
Summoner 180033 from 194785
Summoner Name: T1 Valekey, 'API response: 200
Summoner 180227 from 194785
Summoner Name: BoB Esp0nja, 'API response: 200
Summoner 180421 from 194785
Summoner Name: TheSum41, 'API response: 200


#### Needed Functions for MATCH_V5

##### /lol/match/v5/matches/by-puuid/{puuid}/ids

Response Game IDs for summoner puuid

##### /lol/match/v5/matches/{matchId}

Response Game data for matchid

##### /lol/match/v5/matches/{matchId}/timeline

Response Game time data for matchid

##### MATCH_V5 by PUUID

In [5]:
#100 requests every 120 sec

time_seconds = 10

max_requests = 8

"""Parameters
puuid : unique globally IDs
api_key

type: Filter the list of match ids by the type of match. This filter 
is mutually inclusive of the queue filter meaning 
any match ids returned must match both the queue and type filters.

count: Number of match ids to return.

"""

@sleep_and_retry
@limits(calls= max_requests, period= time_seconds) #period is in seconds 
def fetch_match_V5(puuid, api_key, count = 100, match_type = "ranked"):
    """
    Get Match IDs from Match_V5
    """
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?&count={count}&type={match_type}&api_key={api_key}'
    
    response = requests.get(url)
    
    if response.status_code != 200:
                    raise Exception('API response: {}'.format(response.status_code))
            
    return response.json()

In [14]:
def iterate_match_V5(summoner_v4_lite_df):
    """
    Get all MatchID from Match_V5 API
    """
    #Takes de series I need with the summoner name and converts it into an array to loop
    puuid_name_array = summoner_v4_lite_df["puuid"].dropna().to_numpy()
    
    #generates and empy DF to store and return the data
    complete_df = pd.DataFrame()
    
    #Inicio un contador en 0
    count = 0
    
    #Loops over summoner name
    for i in range(0,len(puuid_name_array)):
              
        response_df = pd.DataFrame((fetch_match_V5(puuid_name_array[i], api_key)))

        complete_df = pd.concat([complete_df,response_df], ignore_index = True)
            
        print(f"Count: {count}")
        
        count = count + 1
                    
        if count % 100 == 0:
            
            print(f"Checkpoint!")#Prints when we send 100 requests)
                
    return complete_df

In [21]:
def save_to_csv_match_V5(df, tier):
    """
    Saves the dataframe obtained from MATCH_V5 into a csv file
    """
    df.to_csv(f'Data/match_V5_{tier}.csv')  
    
    return print(f'\nSaved to Data/summoner_v4_lite_{tier}.csv\n')

In [27]:
%%time

tier_summoner_v4_lite = "GRANDMASTER"

summoner_v4_lite_df = pd.read_csv(f"Data/summoner_v4_lite_{tier_summoner_v4_lite}.csv")

test = iterate_match_V5(summoner_v4_lite_df)

save_to_csv_match_V5(test, tier_summoner_v4_lite)

Count: 0
Count: 1
Count: 2
Count: 3
Count: 4
Count: 5
Count: 6
Count: 7
Count: 8
Count: 9
Count: 10
Count: 11
Count: 12
Count: 13
Count: 14
Count: 15
Count: 16
Count: 17
Count: 18
Count: 19
Count: 20
Count: 21
Count: 22
Count: 23
Count: 24
Count: 25
Count: 26
Count: 27
Count: 28
Count: 29
Count: 30
Count: 31
Count: 32
Count: 33
Count: 34
Count: 35
Count: 36
Count: 37
Count: 38
Count: 39
Count: 40
Count: 41
Count: 42
Count: 43
Count: 44
Count: 45
Count: 46
Count: 47
Count: 48
Count: 49
Count: 50
Count: 51
Count: 52
Count: 53
Count: 54
Count: 55
Count: 56
Count: 57
Count: 58
Count: 59
Count: 60
Count: 61
Count: 62
Count: 63
Count: 64
Count: 65
Count: 66
Count: 67
Count: 68
Count: 69
Count: 70
Count: 71
Count: 72
Count: 73
Count: 74
Count: 75
Count: 76
Count: 77
Count: 78
Count: 79
Count: 80
Count: 81
Count: 82
Count: 83
Count: 84
Count: 85
Count: 86
Count: 87
Count: 88
Count: 89
Count: 90
Count: 91
Count: 92
Count: 93
Count: 94
Count: 95
Count: 96
Count: 97
Count: 98
Count: 99
Checkpoint

In [ ]:
%%time

def loop_match_V5_all_tiers():
    """
    Loops MATCH_V5 for all tiers
    """
    tier_summoner_v4_lite = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "DIAMOND", "MASTER", "GRANDMASTER", "CHALLENGER"]

    for tier in tier_summoner_v4_lite:
        summoner_v4_lite_df = pd.read_csv(f"Data/summoner_v4_lite_{tier_summoner_v4_lite}.csv")

        df = iterate_match_V5(summoner_v4_lite_df)

        save_to_csv_match_V5_lite(df, tier_summoner_v4_lite)

##### MATCH_V5 by MatchID

In [ ]:
import json

import numpy as np

In [ ]:
#100 requests every 120 sec

time_seconds = 10

max_function_calls = 4

@sleep_and_retry
@limits(calls= max_function_calls, period= time_seconds) #period is in seconds 
def fetch_by_matchID(matchID, api_key):
    """
    #Get Match and Match_Timeline from Match_V5
    """
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{matchID}?api_key={api_key}'
    
    response = requests.get(url)
    
    if response.status_code != 200:
                    raise Exception('API response: {}'.format(response.status_code))
            
    url_timeline = f'https://americas.api.riotgames.com/lol/match/v5/matches/{matchID}/timeline?api_key={api_key}'
    
    response_timeline = requests.get(url_timeline)
    
    if response_timeline.status_code != 200:
                    raise Exception('API response: {}'.format(response.status_code))
            
    return (response.json(), response_timeline.json())

In [ ]:
def save_to_csv_matchID(response, tier, matchID):
    """
    Saves the dataframe obtained from MATCH_V5 by MatchID into a .json file
    """
    
    match_data = f'Data/{tier}/Final/{matchID}.json'
    
    with open(match_data, 'w', encoding='utf-8') as f:
        json.dump(response, f, ensure_ascii=False, indent=4)
    
    return None

In [ ]:
def save_to_csv_matchID_timeline(response_timeline, tier, matchID):
    """
    Saves the dataframe obtained from MATCH_V5 timeline by MatchID into a .json file
    """
    
    match_data_timeline = f'Data/{tier}/Timeline/{matchID}_timeline.json'
    
    with open(match_data_timeline, 'w', encoding='utf-8') as f:
        json.dump(response_timeline, f, ensure_ascii=False, indent=4)
    
    return None

In [ ]:
def obtain_matches(tier, max_games, start = 0):
    """
    Iterates matchIDs to obtain match information
    """
    match_ids = pd.read_csv(f'Data/match_V5_{tier}.csv')["0"].drop_duplicates().dropna().to_numpy()
    
    count = 0
    
    for i in range(start,len(match_ids),int(len(match_ids)/max_games)):
        (response, response_timeline) = fetch_by_matchID(match_ids[i], api_key)
        save_to_csv_matchID(response, tier, match_ids[i])
        save_to_csv_matchID_timeline(response_timeline, tier, match_ids[i])
        print(f"Saved {count} matches, last row is {i}") 
        count = count +1
    
    return print(f"Finished looping {len(match_ids)} in {tier}")

In [ ]:
%%time

obtain_matches("GRANDMASTER", 5000, start = 0)